In [2]:
def read_data(path):
    """
    Reads data from nii.gz file
    inputs;
    path: path to the file
    returns; 
    nii: nii object
    data: mri volume as np array
    """
    nii=nib.load(path)
    data=np.asanyarray(nii.dataobj)
    
    return nii, data
        
def save_nii(data, nii, path2save):
    nii2save=nib.Nifti1Image(data, nii.affine)
    nib.save(nii2save, path2save)
    

In [3]:
def read_labels(labels_path):
    """
    Reading SimpleITK-Snap generated labels
    """
    labels=[]
    anats=[]
    with open(labels_path) as f:
        lines = f.readlines()
        for i,line in enumerate(lines):
            if i>14:
                if i==7:
                    anat= ' '.join(line.split('\\cf')[1].split()[8:])
                    anats.append(anat.split("\"")[1])
                    labels.append(int(line.split('\\cf')[1].split()[0]))
                else:
                    anat=' '.join(line.split()[7:])
                    anats.append(anat.split("\"")[1])
                    labels.append(int(line.split()[0]))

    df=pd.DataFrame()
    df["Labels"]=labels
    df["Anatomical Regions"]=anats
    return df

In [ ]:
def read_whs_labels(path):
    """
    Reads the labels from WHS atlas
    """
#     labels_path='./atlas_labels.rtf'
    labels=[]
    anats=[]
    with open(path) as f:
        lines = f.readlines()
        for i,line in enumerate(lines):
            if i>6 and i%2==1:
                if i==7:
                    anat= ' '.join(line.split('\\cf')[1].split()[8:])
    #                 label=[line.split('\\cf')[1].split()[0], anat]
                    anats.append(anat.split("\"")[1])
                    labels.append(int(line.split('\\cf')[1].split()[0]))
    #                 print(label)
                else:
                    anat=' '.join(line.split()[7:])
    #                 label=[line.split()[0], anat]
                    anats.append(anat.split("\"")[1])
                    labels.append(int(line.split()[0]))
    #                 print(label)

    df=pd.DataFrame()
    df["Labels"]=labels
    df["Anatomical Regions"]=anats
    return df

In [ ]:
def read_png(path, scale=cv2.IMREAD_GRAYSCALE):
    img=cv2.imread(path, scale)

    return img

In [ ]:
def sort_directory(directory):
    items = os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items

In [ ]:
def load_imgs(path):
    files=sort_directory(path)
    for idx,file in enumerate(files):
        filename = os.fsdecode(file)
        if filename.endswith(".png"):
            img=read_png(path+filename)
            if idx==0:
                x=np.shape(img)[0]
                y=np.shape(img)[1]
                data=np.zeros((x,y,1))
                data[:,:,0]=img
            else:
                data=np.append(data,img[...,newaxis],2)
    print(np.shape(data))
    return data

In [ ]:
def read_dicom(path):
    ds = pydicom.dcmread(path)
    pixel_array=np.flip(ds.pixel_array)
    img = (np.maximum(pixel_array,0) / pixel_array.max()) * 255.0
    img = np.uint8(img)
    
    return ds, img